# Initialisation

In [700]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#%matplotlib tk #pour Linux

In [701]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [702]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [703]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [704]:
frame_index = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    print(frame_index)

1


# Traitement d'image

In [705]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Sélection du point à tracker

In [706]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze_x = crop_sze_y = 50

Please click on the point you want to select.
Selected point: (716.5779220779222, 619.5649350649351)


In [707]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

# Fit gaussien sur le point sélectionné

In [708]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [709]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [710]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))

    x_position = result.params['x0'].value + maxi[0] - crop_sze_x/2
    y_position = result.params['y0'].value + maxi[1] - crop_sze_y/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [711]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [712]:
def next_frame(frame_i):
    frame_i += 1
    tiff.seek(frame_i)
    original_image = np.array(tiff)
    
    while np.sum(original_image) == 0:
        frame_i += 1
        tiff.seek(frame_i)
        original_image = np.array(tiff)

    return [frame_i, original_image]

Débogueur

In [713]:
def visionneur(frame):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title('Grille Zoomée avec Position')
    plt.colorbar()  
    plt.show()

# Identifier la particule et donner sa nouvelle position

In [714]:
def empreinte_digitale(image, threshold=12, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
    
    seuil=np.max(image)*0.4
    print(seuil)
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [ ]:
def detect_nbre_particles(image, threshold, eps=3, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image)

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
#        if nb_particules > 1:
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    plt.colorbar()  
    plt.legend()
    plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [ ]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=25, eps=3, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity)  
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    print(f'ecrat: {ecart}')
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

In [ ]:
def particle_tracker(image, positions, frame_index, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(image)
    #print(positions[frame_index][0],positions[frame_index][1])

    cropped_img = crop(image, positions[frame_index][0], positions[frame_index][1], crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)
    print(f'coordonnées des {nb_part} particules:{coordonnées}')

    if (nb_part == 0) or (nb_part == []):
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {frame_index}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = positions[frame_index-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({x}, {y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    elif nb_part > 1:
        imperfections, emplacements = [] , []
        for num_particule in range(nb_part):
            x_identifier=positions[frame_index][0] - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=positions[frame_index][1] - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x, crop_sze_y)
            emplacement, ecart = identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale, seuil)
            imperfections.append(ecart)
            emplacements.append(emplacement)
        print(imperfections)
        print(np.argmin(imperfections))
        qualité = np.min(imperfections)
        max_coords = emplacements[np.argmin(imperfections)]  
        print(qualité)
        print(max_coords)
        nouveau_x = x_identifier - crop_sze_x // 2 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 2 + max_coords[1]
    else:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)
    #print(F'coordonnées du max dans le premier crop: ({max_coords[1]},{max_coords[0]})')
    #if frame_index>1:
    #    visionneur(cropped_img)

        nouveau_x = positions[frame_index][0] - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = positions[frame_index][1] - crop_sze_y // 2 + max_coords[0]

    #print(f'(x,y) central du second crop: {nouveau_x, nouveau_y}')
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    #if frame_index>1:
    #    visionneur(second_crop)


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])
    #print(f'écart-type: ({result_fit[1]}, {result_fit[2]})')

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    #print(f'Positions finales de l index {frame_index}: ({x_new}, {y_new}) ')
    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [718]:
position_list = [(x,y), (x,y)]
sigma_list = []
crop_frames = []
big_frames = []
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,crop_sze_x,crop_sze_y))

for _ in range(27 - 1):
    print(f'frame index: {frame_index} et len(positions): {len(position_list)}')
    data = particle_tracker(img, position_list, frame_index, taille_initiale, luminosité_initiale, seuil)
    position_list.append(data[2])
    sigma_list.append(data[3])
    frame_index, img = next_frame(frame_index)
    big_frames.append(img)
    crop_frames.append(data[1])

17.6


C:\Users\maxim\AppData\Local\Temp\ipykernel_12620\3821038158.py:28: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


frame index: 1 et len(positions): 2
coordonnées des 1 particules:[(22.544, 19.76)]
frame index: 2 et len(positions): 3
coordonnées des 1 particules:[(25.75, 36.375)]
frame index: 3 et len(positions): 4
coordonnées des 1 particules:[(24.428571428571427, 30.214285714285715)]
frame index: 4 et len(positions): 5
coordonnées des 1 particules:[(23.962962962962962, 24.37037037037037)]
frame index: 5 et len(positions): 6
coordonnées des 1 particules:[(28.8, 25.25)]
frame index: 6 et len(positions): 7
coordonnées des 1 particules:[(28.75, 33.375)]
frame index: 7 et len(positions): 8
coordonnées des 1 particules:[(28.09090909090909, 36.24242424242424)]
frame index: 8 et len(positions): 9
Aucune particule détectée.
coordonnées des [] particules:[[ 8  8  8 ... 11 10 10]
 [ 8  8  8 ... 10 10 11]
 [ 8  8  8 ... 10 10 11]
 ...
 [ 8  8  8 ...  9 10  9]
 [ 8  8  8 ...  9 10 10]
 [ 9  8  8 ... 10  9 10]]
Please click on the point you want to select.
Selected point: (716.5779220779222, 619.5649350649351)

IndexError: list index out of range

# Résultats

In [ ]:
position_list

[(441.9025974025975, 567.551948051948),
 (441.9025974025975, 567.551948051948),
 (441.5872714824088, 568.5913168655402),
 (441.2407357431485, 567.7405687180908),
 (441.2152557489715, 568.9325811006646),
 (441.4673673118123, 569.3407716993431),
 (441.9374028302384, 569.3606436889775),
 (441.74379141782276, 570.0539052368566),
 (441.616175454853, 571.3094452452121),
 (441.7313177847689, 570.0113240564632),
 (442.0142281329039, 571.049460834359),
 (442.87402700720736, 573.3073044258916),
 (442.5430988722535, 573.2811529943147),
 (441.85200654161935, 573.5775964188261),
 (440.9094038622981, 573.6505209644198),
 (441.87161336982786, 578.9985824771967),
 (441.84823725402, 579.7334846445752),
 (442.1501661165205, 579.8617786578852),
 (442.90248597508696, 580.1847602968634),
 (443.08696981196607, 579.3524263880744),
 (442.58102087952915, 580.503881026763),
 (442.4632463549858, 580.6867390745042),
 (442.47386508136225, 581.2792277022494),
 (443.12295304479204, 582.0272075271115),
 (442.97486559

In [ ]:
sigma_list

[(3.270192918749122, 4.431043381239054),
 (3.0979480431155832, 4.810740694666396),
 (2.8831378310239457, 4.475510479030111),
 (2.616603939117088, 4.729771306006793),
 (2.802100552617834, 4.663950255091089),
 (2.568253605428795, 4.0285793726748444),
 (2.471514270604141, 4.152259541658073),
 (2.9319420045507365, 4.070628381006426),
 (2.700670206288887, 4.2127177518819705),
 (3.0624039939963636, 4.644773809831006),
 (2.8619425705644974, 4.390351154122787),
 (2.8997495695960533, 4.370676042885759),
 (3.4689114130485543, 4.645606006696235),
 (3.216146557675548, 5.103353603075927),
 (2.852139404756468, 4.6724468676730835),
 (2.9209605167300645, 4.236550790195615),
 (2.9266789513322204, 4.520172242747669),
 (2.7725322875695713, 4.397371916795202),
 (2.7410707225813664, 4.899395941306493),
 (3.116833629494117, 4.6441268315830815),
 (2.703607018194674, 4.4226581560628695),
 (2.913579154413197, 4.8052843483714085),
 (3.096148961915269, 4.437438191479247),
 (3.0476152376350827, 4.729363026949236)

In [ ]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()